# Find cases when proteoforms clarify gene-level overlap

### Set up configuration

In [10]:


import config
from lib.graph_database_access import get_query_result
from queries import QUERY_GET_NUM_PROTEOFORMS_PER_PROTEIN

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

config.set_root_wd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initial working directory: C:\git\ProteoformNetworks\src\Python
New working directory: C:\git\ProteoformNetworks


## Select candidate proteins

In [4]:
# Select proteins with multiple proteoforms
df_proteins = get_query_result(QUERY_GET_NUM_PROTEOFORMS_PER_PROTEIN)
df_proteins

,Protein,Proteoforms,NumProteoforms
0,P0CG48,"[[P0CG48], [P0CG48, 00134:152], [P0CG48, 00134...",82
1,P68431,"[[P68431], [P68431, 00046:11, 00047:12, 00064:...",52
2,Q71DI3,"[[Q71DI3], [Q71DI3, 00046:11, 00047:12, 00064:...",48
3,P02452,"[[P02452], [P02452, 00037:null], [P02452, 0003...",36
4,P08123,"[[P08123], [P08123, 00037:null], [P08123, 0003...",36
...,...,...,...
1522,Q13554,"[[Q13554], [Q13554, 00047:287]]",2
1523,Q13555,"[[Q13555], [Q13555, 00047:287]]",2
1524,Q13557,"[[Q13557], [Q13557, 00047:287]]",2
1525,Q13569,"[[Q13569], [Q13569, 01149:330]]",2


In [7]:
print(f"There are {len(df_proteins)} proteins with multiple proteoforms.")

There are 1527 proteins with multiple proteoforms.


In [9]:


selected_proteins = set(df["Protein"].unique())
len(selected_proteins)

# Select proteins that do not participate in many diseases



1527